In [ ]:
# 1. KILL OLD PROCESSES & CLEAN SYSTEM
print("🧹 [1/5] Cleaning up old sessions...")
!pkill -f dockerd || true
!pkill -f cloudflared || true
!pkill -f npx || true
!rm -rf /etc/apt/sources.list.d/r2u.list

# 2. INSTALL & FORCE START DOCKER
print("⚡ [2/5] Starting Engine...")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb -qq > /dev/null
import os, subprocess, time, threading
subprocess.Popen(['dockerd', '--host=unix:///var/run/docker.sock'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(15)

# 3. BOOT ANDROID & SYNC ADB
print("📱 [3/5] Booting Android...")
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest
time.sleep(5)
!adb connect localhost:5555

# 4. WEB PORTAL
print("🌐 [4/5] Opening Portal...")
!npx -y ws-scrcpy-docker --port 7860 > /dev/null 2>&1 &

# 5. CLOUDFLARE LINK
print("🔗 [5/5] Generating Link (Wait 20s)...")
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

def run_tunnel():
    os.system("./cloudflared tunnel --no-autoupdate --url http://localhost:7860")

threading.Thread(target=run_tunnel, daemon=True).start()
time.sleep(20)
print("\n🚀 ALL SYSTEMS GO!")
print("Look for the 'trycloudflare.com' link in the logs below.")